_Updated 200220_

In [ ]:
import math
import numpy as np
import scipy as sp
from scipy import integrate
from scipy import interpolate
from scipy.optimize import minimize
import matplotlib.pyplot as plt

Parameters

In [32]:
# Temperature
temp = 298

# Fermi Level
# The Fermi energy printed out by QE (in the DOS output file) is 7.770. 
# The output is truncated and it turns this is too inaccurate to obtain the correct electron number. 
# We determined the Fermi energy to higher accuracy (by imposing the correct electron number = 3*256 = 768).
# There are 3 electrons per atom due to the choice of pseudopotential. 
# CAUTION: Need to define an accurate Fermi energy for each snapshot. 
fermi_energy = 7.770345

# Boltzmann's constant
k = 8.617333262145e-5

# Conversion factor from Rydberg to eV
Ry2eV = 13.6056980659

# Gaussian smearing in QE-DOS calculations
# taken from QE-DOS input file
sigma_qe = 0.032

Load eigenvalues and DOS from QE output for two snapshots

In [3]:
# filepath: blake.sandia.gov:/home/acangi/q-e_calcs/Al/datasets/vasp_econ_snapshots/298K/2.699g/170726180545.0/100Ry_k333
## Snapshot 0: Eigenvalues (from PW std output file, slurm-1006575.out)
### rows: band index, row i: eigs[i , :]
### cols: k points,   col j: eigs[: , j]
eigs_qe_00 = np.loadtxt('snap_0/EIGS', delimiter=',')
k_weights_qe_00 = np.loadtxt('snap_0/k_weights', delimiter=',')
## DOS
dos_qe_00 = np.loadtxt('snap_0/Al.dos', skiprows=1)
## Snapshot 1: Eigenvalues (from PW std output file, slurm-1006846.out)
### rows: band index, row i: eigs[i , :]
### cols: k points,   col j: eigs[: , j]
eigs_qe_01 = np.loadtxt('snap_1/EIGS', delimiter=',')
k_weights_qe_01 = np.loadtxt('snap_1/k_weights', delimiter=',')
## DOS
dos_qe_01 = np.loadtxt('snap_1/Al.dos', skiprows=1)

Define functions

In [4]:
# Fermi-Dirac distribution function
def fd_function(energy, eF, t):                                                                                                 
    return 1.0 / (1.0 + np.exp((energy - eF) / (k * t)))

In [5]:
# Define Gaussian
## Note: Gaussian without factor of 1/sqrt(2)
def gaussian(en, eF, sigma):
    result = 1.0/np.sqrt(np.pi*sigma**2)*np.exp(-1.0*((en-eF)/sigma)**2)
    return result

Here we define functions which compute reference values of the electron number and band energy = sum of eigenvalues.
Recall the some definitions:

The electron number is defined as
$$ N = \sum_{k,i} w_{k,i}\, f(\epsilon(k,i) - \mu)\,,$$
the band energy is defined as
$$ E_b = \sum_{k,i} w_{k,i}\, \epsilon(k,i) f(\epsilon(k,i) - \mu)\,,$$ 
where $i$ denotes the band, $k$ the k point, $w_{k,i}$ the weight of the k point, $\epsilon(k,i)$ the eigenvalue for a given k point and band, $\mu$ the chemical potential (which equals the Fermi energy at zero temperature), and $f$ the Fermi-Dirac distribution function (which truncates the sum at the chemical potential). 

In [6]:
# Function generating electron number from sum of eigenvalues
def gen_ENUM(k_weights, array_eigs):
    # input:
    ## k_weights: weights of k-point summation (taken from QE output)
    ## array_eigs: array[dim_bnd, dim_k] containing eigenvalues (\epsilon_{i,k})
    # output:
    dim_bnd = len((array_eigs[: , 0]))
    dim_k = len((array_eigs[0 , :]))
    ra_fd = fd_function(array_eigs, eF=fermi_energy, t=temp)
    enum = 0.0
    for idx_bnd in range(dim_bnd):
        for idx_k in range(dim_k):
            # Sum the Gaussians over idx_band and idx_k
            enum += k_weights[idx_k]*ra_fd[idx_bnd , :][idx_k]  
    return enum

In [7]:
# Function generating sum of eigenvalues = band energy
def gen_EBAND(k_weights, array_eigs):
    # input:
    ## k_weights: weights of k-point summation (taken from QE output)
    ## array_eigs: array[dim_bnd, dim_k] containing eigenvalues (\epsilon_{i,k})
    # output:
    ## eband
    dim_bnd = len((array_eigs[: , 0]))
    dim_k = len((array_eigs[0 , :]))
    ra_fd = fd_function(array_eigs, eF=fermi_energy, t=temp)
    eband = 0.0
    for idx_bnd in range(dim_bnd):
        for idx_k in range(dim_k):
            # Sum the Gaussians over idx_band and idx_k
            eband += k_weights[idx_k]*array_eigs[idx_bnd , :][idx_k]*ra_fd[idx_bnd , :][idx_k]  
    return eband

Compute the reference values for the electron number and the band energy by directly summing over the eigenvalues.
It turns out the value of the band energy obtained from QE below is not correct.

In [31]:
# Reference values of electron number and band energy
## Snapshot 0
enum_ref_00  = gen_ENUM(k_weights_qe_00, eigs_qe_00)
eband_ref_00 = gen_EBAND(k_weights_qe_00, eigs_qe_00)
print("SNAPSHOT 0")
print("Electron number:", enum_ref_00)
print("Band energy [eV]:", eband_ref_00)
print(" ")

## Snapshot 1 #Need to determine more accurate Fermi energy for snaphot2
#fermi_energy = 7.7733618
enum_ref_01  = gen_ENUM(k_weights_qe_01, eigs_qe_01)
eband_ref_01 = gen_EBAND(k_weights_qe_01, eigs_qe_01)
print("SNAPSHOT 1")
print("Electron number:", enum_ref_01)
print("Band energy [eV]:", eband_ref_01)

SNAPSHOT 0
Electron number: 768.3800699587922
Band energy [eV]: 2598.726246188354
 
SNAPSHOT 1
Electron number: 768.0000014009643
Band energy [eV]: 2598.579547828817


Alternatively, the electron number and band energy can be by aid of the density of states. This is useful, because, eventually, we will predict the local DOS/DOS from ML.

In terms of the DOS, they are defined as 

$$N = \int_{-\infty}^{\infty} d\epsilon\ D(\epsilon)\, f(\epsilon)$$
and
$$E_b = \int_{-\infty}^{\infty} d\epsilon\ D(\epsilon)\, f(\epsilon)\, \epsilon\,,$$


where $\epsilon$ denotes the energy as a continuous variable, $D(\epsilon)$ the DOS, $f(\epsilon)$ the Fermi-Dirac distribution function.

The explicit defintion of the DOS is given as a sum of $\delta$-functions over the spectrum of eigenvalues.
$$ D(\epsilon) = \sum_i \sum_k w_k\, \delta(\epsilon-\epsilon(i,k))$$

Commonly (in particular, for the purposes of visualization), the $\delta$-functions are represented by particular choice of functions (for example, Gaussians)
$$\delta(\epsilon-\epsilon_{ik}) = \frac{1}{\sqrt{\pi\sigma^2}}\exp{\left[-\left(\frac{\epsilon-\epsilon_{ik}}{\sigma}\right)^2\right]}$$ 
where $\sigma$ denotes the width.

In [ ]:
# Function generating DOS sum over eigenvalues
def gen_DOS(k_weights, array_en, array_eigs, sigma):
    # input:
    ## k_weights: weights of k-point summation (taken from QE output)
    ## array_en: energy grid [eV]
    ## array_eigs: array[dim_bnd, dim_k] containing eigenvalues (\epsilon_{i,k})
    ## sigma: width of Gaussian [eV]
    # output:
    ## array_dos: ra_dos
    ## array_dos_contr: ra_dos_ik (terms for each i,k)
    dim_bnd = len((array_eigs[: , 0]))
    dim_k = len((array_eigs[0 , :]))
    ra_en = array_en #dos_qe[: , 0]       # energy grid (same as QE-DOS input/output) 
    ra_dos_ik = [[] for i in range(dim_bnd)]
    ra_dos = np.zeros(len(array_en)) #create empty array
    for idx_bnd in range(dim_bnd):
        for idx_k in range(dim_k):
            ra_dos_ik[idx_bnd].append(gaussian(ra_en, array_eigs[idx_bnd , :][idx_k], sigma))
            # Sum the Gaussians over idx_band and idx_k
            ra_dos += k_weights[idx_k]*ra_dos_ik[idx_bnd][idx_k]
    return ra_dos #, ra_dos_ik

In [ ]:
# Function generating electron number from DOS
## Integrate DOS*FD to obtain band energy
def gen_enumFromDOS(k_weights, array_en, array_eigs, sigma):
    # input:
    ## k_weights: weights of k-point summation (taken from QE output)
    ## array_eigs: array[dim_bnd, dim_k] containing eigenvalues (\epsilon_{i,k})
    ## sigma: width of Gaussian [eV]
    # output:
    ## enum
    ra_fd = fd_function(array_en, eF=fermi_energy, t=temp)
    #ra_dos, ra_dos_ik = gen_DOS(k_weights_qe, array_en, eigs_qe, sigma)
    ra_dos = gen_DOS(k_weights, array_en, array_eigs, sigma)
    enum = sp.integrate.trapz(ra_dos*ra_fd, array_en)
    return enum

In [ ]:
# Function generating band energy from DOS
## Integrate DOS*E*FD to obtain band energy
def gen_ebandFromDOS(k_weights, array_en, array_eigs, sigma):
    # input:
    ## k_weights: weights of k-point summation (taken from QE output)
    ## array_en: energy grid [eV]
    ## array_eigs: array[dim_bnd, dim_k] containing eigenvalues (\epsilon_{i,k})
    ## sigma: width of Gaussian [eV]
    # output:
    ## eband [Ry]
    ra_fd = fd_function(array_en, eF=fermi_energy, t=temp)
    #ra_dos, ra_dos_ik = gen_DOS(k_weights_qe, array_en, eigs_qe, sigma)
    ra_dos = gen_DOS(k_weights, array_en, array_eigs, sigma)
    eband = sp.integrate.simps(ra_dos*array_en*ra_fd, array_en)
    #Convert from eV to Ry for comparison with QE output
    eband_Ry = eband/Ry2eV
    return eband_Ry

Compute total DOS and compare with QE output

In [ ]:
# Generate data
ra_en_00 = dos_qe_00[: , 0]
ra_dos_00 = gen_DOS(k_weights_qe_00, ra_en_00, eigs_qe_00, sigma=sigma_qe*Ry2eV )
        
# Plot data        
plt.figure(figsize=[8,6])
ax = plt.subplot(1,1,1)
ax.set_xlabel(r'E [eV]')
ax.set_ylabel(r'D(E)')
plt.rcParams.update({'font.size': 22})

ax.plot(dos_qe_00[: , 0],  dos_qe_00[: , 1], linestyle='-',  linewidth=3, color='black', label='QE-DOS')
ax.plot(dos_qe_00[: , 0] , ra_dos_00,        linestyle='--', linewidth=3, color='red',  label='computed')

# Legend
ax.legend(loc='upper right',frameon=False, bbox_to_anchor=(1.5, 1))
plt.show()

We recover the QE-DOS result by using the same parameters as in the input for computing the DOS.
Now we can go ahead and investigate different energy grids and smearing values in order to improve upon the band energy and achieve better agreement with the band-energy output of QE.

**CAUTION: This is not correct. The direct sum over eigenvalues (see analysis above) does not yield the same value for the band energy.** 
The QE output of snapshot 0 is given at  ```blake.sandia.gov:/home/acangi/q-e_calcs/Al/datasets/vasp_econ_snapshots/298K/2.699g/170726180545.0/100Ry_k333```. 
QE prints the one-electron energy in the standard output, together with all the other energy contributions, e.g., 
```
The total energy is the sum of the following terms:                                                  


     one-electron contribution =     737.82754675 Ry
     hartree contribution      =       4.77073244 Ry
     xc contribution           =    -554.09988814 Ry
     ewald contribution        =   -1375.56724973 Ry
     smearing contrib. (-TS)   =      -0.02019845 Ry
```
However, what QE prints as the "one-electron contribution" is not the sum of the eigenvalues, but instead (see source code ```~/PW/src/electrons.f90``` lines 638-640)

$$\text{one-electron contribution} = \sum_i \epsilon_i - (E_h + E_{xc})$$

In order to correctly compare the band energy obtained from integrating the DOS with the QE output we need to add the hartree and exchange-correlation contributions to the one-electron contribution. 

**However, this does not agree with the reference value of the band energy we obtain above by explicitly summing over the eigenvalues (which we know is the correct band energy.)**

In [ ]:
eband_qe = 737.82754675+4.77073244-554.09988814
print("Band energy from QE output [Ry]:", eband_qe)
print("Reference value of band energy (correct) [Ry]:", eband_ref_00/Ry2eV)

Below we analyze the behavior of the predicted electron number and band energy with respect to the choice of energy bin and smearing. One important question to answer is whether we can recover an accurate band energy (error < 10 meV) for a relatively smooth DOS which will be more suited for ML.

## Snapshot 0:

In [ ]:
# Define energy bin
#ewidth=dos_qe[: , 0][1]-dos_qe[: , 0][0]
ewidth = (dos_qe_00[: , 0][1]-dos_qe_00[: , 0][0])*0.5
print(ewidth)
# Define smearing
ra_sigma_00 = np.linspace(0.5*ewidth, 5*ewidth, 11)
ra_enum_00 = np.zeros(len(ra_sigma_00))
ra_eband_00 = np.zeros(len(ra_sigma_00))
ra_en = np.linspace(dos_qe_00[0, 0], dos_qe_00[-1, 0], int((dos_qe_00[-1, 0]-dos_qe_00[0, 0])/ewidth)) 
for i in range(len(ra_sigma_00)):
    print("i:", i)
    # Generate electron number 
    ra_enum_00[i] = gen_enumFromDOS(k_weights_qe_00, ra_en, eigs_qe_00, sigma=ra_sigma_00[i])
    # Generate band energy
    ra_eband_00[i] = gen_ebandFromDOS(k_weights_qe_00, ra_en , eigs_qe_00, sigma=ra_sigma_00[i])
print(ra_en[1]-ra_en[0])
print(ra_enum_00)
print(ra_eband_00)

In [ ]:
# Plot data        
plt.figure(figsize=[8,6])
ax = plt.subplot(1,1,1)
ax.set_xlabel(r'Width of smearing [Ry]')
#ax.set_ylabel(r'D(E)')
plt.rcParams.update({'font.size': 22})

#ax.set_ylim(767.95, 768.05)

ax.hlines(enum_ref_00, ra_sigma_00[0], ra_sigma_00[-1], linewidth=3)
ax.plot(ra_sigma_00,  ra_enum_00, linestyle='-',  linewidth=3, color='blue', label='Electron number')

# Legend
ax.legend(loc='upper right',frameon=False, bbox_to_anchor=(1.5, 1))
plt.show()

In [ ]:
# Plot data        
plt.figure(figsize=[8,6])
ax = plt.subplot(1,1,1)
ax.set_xlabel(r'Width of smearing [Ry]')
#ax.set_ylabel(r'D(E)')
plt.rcParams.update({'font.size': 22})


ax.hlines(eband_ref_00/Ry2eV, ra_sigma_00[0], ra_sigma_00[-1], linewidth=3)
ax.scatter(ra_sigma_00,  ra_eband_00, linestyle='-',  linewidth=3, color='red', label='Band energy')

# Legend
ax.legend(loc='upper right',frameon=False, bbox_to_anchor=(1.5, 1))
plt.show()
print("Error in band energy [Ry/atom]", np.min(abs(ra_eband_00-eband_ref_00/Ry2eV))/256)

Generate the corresponding DOS

In [ ]:
ra_dos_00 = gen_DOS(k_weights_qe_00, ra_en, eigs_qe_00, sigma=ra_sigma_00[1] )

In [ ]:
# Plot data        
plt.figure(figsize=[8,6])
ax = plt.subplot(1,1,1)
ax.set_xlabel(r'E [eV]')
ax.set_ylabel(r'D(E)')
plt.rcParams.update({'font.size': 22})

ax.plot(ra_en           , ra_dos_00,        linestyle='-',  linewidth=1, color='red',   label='computed (adjusted width)')
ax.plot(dos_qe_00[: , 0], dos_qe_00[: , 1], linestyle='-',  linewidth=1, color='black', label='QE-DOS')

# Legend
ax.legend(loc='upper right',frameon=False, bbox_to_anchor=(2, 1))
plt.show()

## Snapshot 1:

In [ ]:
# Define energy bin
#ewidth=dos_qe[: , 0][1]-dos_qe[: , 0][0]
ewidth = (dos_qe_01[: , 0][1]-dos_qe_01[: , 0][0])*0.5
print(ewidth)
# Define smearing
ra_sigma_01 = np.linspace(0.5*ewidth, 5*ewidth, 11)
ra_enum_01 = np.zeros(len(ra_sigma_01))
ra_eband_01 = np.zeros(len(ra_sigma_01))
ra_en_01 = np.linspace(dos_qe_01[0, 0], dos_qe_01[-1, 0], int((dos_qe_01[-1, 0]-dos_qe_01[0, 0])/ewidth)) 
for i in range(len(ra_sigma_01)):
    print("i:", i)
    # Generate electron number 
    ra_enum_01[i] = gen_enumFromDOS(k_weights_qe_01, ra_en, eigs_qe_01, sigma=ra_sigma_01[i])
    # Generate band energy
    ra_eband_01[i] = gen_ebandFromDOS(k_weights_qe_01, ra_en_01 , eigs_qe_01, sigma=ra_sigma_01[i])
print(ra_en_01[1]-ra_en_01[0])
print(ra_enum_01)
print(ra_eband_01)

This illustrates the issue with Gaussian smearing. The smearing width differs between the different snapshots. This means we cannot choose a fixed smearing width and obtain high accuracy in the band energy throughout a priori (i.e. without knowing the true value of the band energy). However, this might be fine, since we need to choose the smearing width only for the generation of training data. It might be somewhat inconvenient, but for each snapshot in the training data we can find the corresponding smearing width which will yield a band energy up to a target accuracy.

# Scratchpad

To do
* Different representation of the $\delta$ function, for example Marzari-Vanderbilt (MV).

In [ ]:
# Define MV representation of delta function
def MV(en, mu, sigma):
    x = (mu-en)/sigma
    result = 1.0/np.sqrt(np.pi)*(2.0-np.sqrt(2)*x)*np.exp(-1.0*(x-(1.0/np.sqrt(2)))**2)
    return result

In [ ]:
# Sanity check of smearing functions
# Generate data
ra_en = np.linspace(7.,9.,601)
ra_gaussian = gaussian(ra_en, eF=fermi_energy, sigma=sigma_qe)
ra_MV = MV(ra_en, mu=fermi_energy, sigma=sigma_qe*2)
        
# Plot data        
plt.figure(figsize=[8,6])
ax = plt.subplot(1,1,1)
ax.set_xlabel(r'E [eV]')
ax.set_ylabel(r'D(E)')
plt.rcParams.update({'font.size': 22})

ax.plot(ra_en, ra_MV, linestyle='-',  linewidth=3, color='red', label='MV')
#ax.plot(ra_en, ra_gaussian, linestyle='-',  linewidth=3, color='blue', label='gaussian')

# Legend
ax.legend(loc='upper right',frameon=False, bbox_to_anchor=(1.5, 1))
plt.show()

In [ ]:
# Function generating DOS from eigenvalues
def gen_DOS_ST(k_weights, array_en, array_eigs, sigma, smearing_type):
    # input:
    ## k_weights: weights of k-point summation (taken from QE output)
    ## array_en: energy grid [eV]
    ## array_eigs: array[dim_bnd, dim_k] containing eigenvalues (\epsilon_{i,k})
    ## sigma: width of Gaussian [eV]
    # output:
    ## array_dos: ra_dos
    ## array_dos_contr: ra_dos_ik (terms for each i,k)
    dim_bnd = len((array_eigs[: , 0]))
    dim_k = len((array_eigs[0 , :]))
    ra_en = array_en #dos_qe[: , 0]       # energy grid (same as QE-DOS input/output) 
    ra_dos_ik = [[] for i in range(dim_bnd)]
    ra_dos = np.zeros(len(array_en)) #create empty array
    for idx_bnd in range(dim_bnd):
        for idx_k in range(dim_k):
            if (smearing_type == 1):
                smearing = gaussian(en=array_en, eF=array_eigs[idx_bnd , :][idx_k], sigma=sigma)
            elif (smearing_type == 2):
                smearing = MV(en=array_en, mu=array_eigs[idx_bnd , :][idx_k], sigma=sigma)
            else:
                print("Error, choose valid smearing function.")
            ra_dos_ik[idx_bnd].append(smearing)
            # Sum the Gaussians over idx_band and idx_k
            ra_dos += k_weights[idx_k]*ra_dos_ik[idx_bnd][idx_k]
    return ra_dos #, ra_dos_ik

In [ ]:
# Generate data
ra_en = dos_qe[: , 0]
ra_dos = gen_DOS_ST(k_weights_qe, ra_en, eigs_qe, sigma=sigma_qe*Ry2eV,smearing_type=2)

# Plot data        
plt.figure(figsize=[8,6])
ax = plt.subplot(1,1,1)
ax.set_xlabel(r'E [eV]')
ax.set_ylabel(r'D(E)')
plt.rcParams.update({'font.size': 22})

ax.plot(dos_qe[: , 0],  dos_qe[: , 1], linestyle='-',  linewidth=3, color='black', label='QE-DOS')
ax.plot(dos_qe[: , 0] , ra_dos,        linestyle='--', linewidth=3, color='red',  label='computed')

# Legend
ax.legend(loc='upper right',frameon=False, bbox_to_anchor=(1.5, 1))
plt.show()

In [ ]:
# Function generating band energy from DOS
## Integrate DOS*E*FD to obtain band energy
def gen_eband_ST(k_weights, array_en, array_eigs, sigma, smearing_type):
    # input:
    ## k_weights: weights of k-point summation (taken from QE output)
    ## array_en: energy grid [eV]
    ## array_eigs: array[dim_bnd, dim_k] containing eigenvalues (\epsilon_{i,k})
    ## sigma: width of Gaussian [eV]
    # output:
    ## array_dos: ra_dos
    ## array_dos_contr: ra_dos_ik (terms for each i,k)
    ra_fd = fd_function(array_en, eF=fermi_energy, t=temp)
    #ra_dos, ra_dos_ik = gen_DOS(k_weights_qe, array_en, eigs_qe, sigma)
    ra_dos = gen_DOS_ST(k_weights_qe, array_en, eigs_qe, sigma, smearing_type)
    eband = sp.integrate.trapz(ra_dos*array_en*ra_fd, array_en)
    #Convert from eV to Ry for comparison with QE output
    eband_Ry = eband/Ry2eV
    return eband_Ry

In [ ]:
# Generate band energy
emin = dos_qe[: , 0][0]-1
emax = dos_qe[: , 0][-1]+1
ra_en = np.linspace(emin, emax, int(len(dos_qe[: , 0])))
sigma_mod = sigma_qe*30.56825
eband = gen_eband_ST(k_weights_qe, ra_en , eigs_qe, sigma=sigma_mod, smearing_type=2)
print("smearing width {0} eV ({1} Ry)".format(sigma_mod, sigma_mod/Ry2eV))
print("Band energy {0} Ry)".format(eband))

In [ ]:
# Error in band energy (due to discretization of the energy grid in DOS calculation and choice of smearing width)
eband_error = eband-eband_qe
print("Error in Rydberg", eband_error)
print("Error in eV", eband_error*Ry2eV)

In [ ]:
# Generate data
ra_dos = gen_DOS_ST(k_weights_qe, ra_en, eigs_qe, sigma=sigma_mod, smearing_type=2)
        
# Plot data        
plt.figure(figsize=[8,6])
ax = plt.subplot(1,1,1)
ax.set_xlabel(r'E [eV]')
ax.set_ylabel(r'D(E)')
plt.rcParams.update({'font.size': 22})

ax.plot(dos_qe[: , 0],  dos_qe[: , 1], linestyle='-',  linewidth=3, color='black', label='QE-DOS')
ax.plot(dos_qe[: , 0] , ra_dos,        linestyle='-',  linewidth=3, color='red',  label='computed (adjusted width)')

# Legend
ax.legend(loc='upper right',frameon=False, bbox_to_anchor=(2, 1))
plt.show()